# IBKR (Interactive Brokers) API Functions

This notebook contains comprehensive functions for interacting with the Interactive Brokers API using the TWS (Trader Workstation) API.

## Features:
- Connection management
- Market data retrieval (real-time and historical)
- Order placement and management
- Portfolio and account information
- Position management
- Contract details and search

## Prerequisites:
1. Install Interactive Brokers TWS or IB Gateway
2. Enable API connections in TWS/Gateway
3. Install required Python packages: `pip install ib_insync pandas numpy`

## Important Notes:
- Paper trading is recommended for testing
- Ensure proper risk management
- Always test in paper trading before live trading

# Importing Required Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import datetime as dt
import time
import logging
import threading
from typing import Optional, List, Dict, Any, Union
from dataclasses import dataclass
from enum import Enum

# Try to import ib_insync, fallback to manual implementation if not available
try:
    from ib_insync import *
    IB_INSYNC_AVAILABLE = True
    print("ib_insync imported successfully")
except ImportError:
    IB_INSYNC_AVAILABLE = False
    print("ib_insync not available. Using manual implementation.")
    print("Install with: pip install ib_insync")

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration and Constants

In [ ]:
# IBKR Connection Configuration
DEFAULT_HOST = '127.0.0.1'
DEFAULT_TWS_PORT = 7497  # TWS Paper Trading
DEFAULT_GATEWAY_PORT = 4002  # Gateway Paper Trading
LIVE_TWS_PORT = 7496  # TWS Live Trading
LIVE_GATEWAY_PORT = 4001  # Gateway Live Trading

# Default client ID
DEFAULT_CLIENT_ID = 1

# Market data types
class MarketDataType(Enum):
    LIVE = 1
    FROZEN = 2
    DELAYED = 3
    DELAYED_FROZEN = 4

# Order types
class OrderType(Enum):
    MARKET = 'MKT'
    LIMIT = 'LMT'
    STOP = 'STP'
    STOP_LIMIT = 'STP LMT'
    TRAILING_STOP = 'TRAIL'
    RELATIVE = 'REL'
    MARKET_ON_CLOSE = 'MOC'
    LIMIT_ON_CLOSE = 'LOC'

# Time in force
class TimeInForce(Enum):
    DAY = 'DAY'
    GOOD_TILL_CANCEL = 'GTC'
    IMMEDIATE_OR_CANCEL = 'IOC'
    FILL_OR_KILL = 'FOK'
    GOOD_TILL_DATE = 'GTD'
    MARKET_ON_OPEN = 'OPG'

# Data Structures

In [ ]:
@dataclass
class IBKRConfig:
    """Configuration for IBKR connection"""
    host: str = DEFAULT_HOST
    port: int = DEFAULT_TWS_PORT
    client_id: int = DEFAULT_CLIENT_ID
    timeout: int = 30
    readonly: bool = False
    
@dataclass
class ContractDetails:
    """Contract details structure"""
    symbol: str
    sec_type: str
    exchange: str
    currency: str
    expiry: str = ''
    strike: float = 0.0
    right: str = ''
    multiplier: str = ''
    
@dataclass
class OrderDetails:
    """Order details structure"""
    action: str  # BUY or SELL
    quantity: int
    order_type: str
    limit_price: float = 0.0
    stop_price: float = 0.0
    time_in_force: str = TimeInForce.DAY.value
    good_till_date: str = ''
    
@dataclass
class MarketData:
    """Market data structure"""
    symbol: str
    bid: float
    ask: float
    last: float
    volume: int
    timestamp: dt.datetime

# Main IBKR Functions Class

In [ ]:
class IBKR_Functions:
    """
    Comprehensive IBKR API Functions Class
    
    This class provides a complete interface to the Interactive Brokers API,
    including connection management, market data, order management, and portfolio functions.
    """
    
    def __init__(self, config: IBKRConfig = None):
        """Initialize IBKR Functions"""
        self.config = config or IBKRConfig()
        self.ib = None
        self.connected = False
        self.market_data_subscriptions = {}
        self.order_status_callbacks = {}
        
    def connect(self, host: str = None, port: int = None, client_id: int = None) -> bool:
        """
        Connect to Interactive Brokers TWS or Gateway
        
        Args:
            host: TWS/Gateway host address
            port: TWS/Gateway port
            client_id: Unique client identifier
            
        Returns:
            bool: True if connected successfully
        """
        try:
            if not IB_INSYNC_AVAILABLE:
                logger.warning("ib_insync not available. Please install: pip install ib_insync")
                return False
                
            # Use provided parameters or defaults
            host = host or self.config.host
            port = port or self.config.port
            client_id = client_id or self.config.client_id
            
            # Initialize IB connection
            self.ib = IB()
            self.ib.connect(host, port, clientId=client_id, timeout=self.config.timeout)
            
            # Set up event handlers
            self.ib.orderStatusEvent += self._on_order_status
            self.ib.execDetailsEvent += self._on_execution
            self.ib.errorEvent += self._on_error
            
            self.connected = True
            logger.info(f"Connected to IBKR at {host}:{port} with client ID {client_id}")
            return True
            
        except Exception as e:
            logger.error(f"Failed to connect to IBKR: {str(e)}")
            self.connected = False
            return False
    
    def disconnect(self):
        """Disconnect from IBKR"""
        if self.ib and self.connected:
            self.ib.disconnect()
            self.connected = False
            logger.info("Disconnected from IBKR")
    
    def is_connected(self) -> bool:
        """Check if connected to IBKR"""
        return self.connected and self.ib.isConnected() if self.ib else False
    
    def _on_order_status(self, trade):
        """Handle order status updates"""
        logger.info(f"Order status update: {trade}")
        
    def _on_execution(self, trade, fill):
        """Handle execution updates"""
        logger.info(f"Execution: {fill}")
        
    def _on_error(self, reqId, errorCode, errorString, contract):
        """Handle error messages"""
        logger.error(f"Error {errorCode}: {errorString}")
    
    # Market Data Functions
    def get_market_data(self, contract_details: ContractDetails, data_type: MarketDataType = MarketDataType.DELAYED) -> Optional[MarketData]:
        """
        Get real-time market data for a contract
        
        Args:
            contract_details: Contract details
            data_type: Market data type (live, delayed, etc.)
            
        Returns:
            MarketData object or None
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return None
            
        try:
            # Create contract
            contract = self._create_contract(contract_details)
            
            # Request market data
            self.ib.reqMarketDataType(data_type.value)
            ticker = self.ib.reqMktData(contract, '', False, False)
            self.ib.sleep(2)  # Wait for data
            
            # Extract market data
            if ticker.bid and ticker.ask and ticker.last:
                return MarketData(
                    symbol=contract_details.symbol,
                    bid=ticker.bid,
                    ask=ticker.ask,
                    last=ticker.last,
                    volume=ticker.volume or 0,
                    timestamp=dt.datetime.now()
                )
            return None
            
        except Exception as e:
            logger.error(f"Error getting market data: {str(e)}")
            return None
    
    def get_historical_data(self, contract_details: ContractDetails, duration: str = '1 Y', 
                          bar_size: str = '1 day', what_to_show: str = 'TRADES') -> pd.DataFrame:
        """
        Get historical data for a contract
        
        Args:
            contract_details: Contract details
            duration: Duration string (e.g., '1 Y', '6 M', '30 D')
            bar_size: Bar size (e.g., '1 day', '1 hour', '5 mins')
            what_to_show: Data type ('TRADES', 'MIDPOINT', 'BID', 'ASK')
            
        Returns:
            DataFrame with historical data
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return pd.DataFrame()
            
        try:
            contract = self._create_contract(contract_details)
            
            # Request historical data
            bars = self.ib.reqHistoricalData(
                contract=contract,
                endDateTime='',
                durationStr=duration,
                barSizeSetting=bar_size,
                whatToShow=what_to_show,
                useRTH=True,
                formatDate=1
            )
            
            # Convert to DataFrame
            if bars:
                df = util.df(bars)
                df['symbol'] = contract_details.symbol
                return df
            return pd.DataFrame()
            
        except Exception as e:
            logger.error(f"Error getting historical data: {str(e)}")
            return pd.DataFrame()
    
    # Order Management Functions
    def place_order(self, contract_details: ContractDetails, order_details: OrderDetails) -> Optional[int]:
        """
        Place an order
        
        Args:
            contract_details: Contract details
            order_details: Order details
            
        Returns:
            Order ID or None
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return None
            
        try:
            contract = self._create_contract(contract_details)
            order = self._create_order(order_details)
            
            # Place order
            trade = self.ib.placeOrder(contract, order)
            
            logger.info(f"Order placed: {trade.order.orderId}")
            return trade.order.orderId
            
        except Exception as e:
            logger.error(f"Error placing order: {str(e)}")
            return None
    
    def cancel_order(self, order_id: int) -> bool:
        """
        Cancel an order
        
        Args:
            order_id: Order ID to cancel
            
        Returns:
            bool: True if successful
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return False
            
        try:
            # Find the order
            trades = self.ib.openTrades()
            for trade in trades:
                if trade.order.orderId == order_id:
                    self.ib.cancelOrder(trade.order)
                    logger.info(f"Order cancelled: {order_id}")
                    return True
            
            logger.warning(f"Order not found: {order_id}")
            return False
            
        except Exception as e:
            logger.error(f"Error cancelling order: {str(e)}")
            return False
    
    def get_open_orders(self) -> pd.DataFrame:
        """
        Get all open orders
        
        Returns:
            DataFrame with open orders
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return pd.DataFrame()
            
        try:
            trades = self.ib.openTrades()
            orders_data = []
            
            for trade in trades:
                orders_data.append({
                    'order_id': trade.order.orderId,
                    'symbol': trade.contract.symbol,
                    'action': trade.order.action,
                    'quantity': trade.order.totalQuantity,
                    'order_type': trade.order.orderType,
                    'limit_price': trade.order.lmtPrice,
                    'status': trade.orderStatus.status,
                    'filled': trade.orderStatus.filled,
                    'remaining': trade.orderStatus.remaining
                })
            
            return pd.DataFrame(orders_data)
            
        except Exception as e:
            logger.error(f"Error getting open orders: {str(e)}")
            return pd.DataFrame()
    
    # Portfolio and Account Functions
    def get_portfolio_positions(self) -> pd.DataFrame:
        """
        Get current portfolio positions
        
        Returns:
            DataFrame with portfolio positions
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return pd.DataFrame()
            
        try:
            portfolio = self.ib.portfolio()
            positions_data = []
            
            for position in portfolio:
                positions_data.append({
                    'symbol': position.contract.symbol,
                    'sec_type': position.contract.secType,
                    'exchange': position.contract.exchange,
                    'currency': position.contract.currency,
                    'position': position.position,
                    'market_price': position.marketPrice,
                    'market_value': position.marketValue,
                    'average_cost': position.averageCost,
                    'unrealized_pnl': position.unrealizedPNL,
                    'realized_pnl': position.realizedPNL
                })
            
            return pd.DataFrame(positions_data)
            
        except Exception as e:
            logger.error(f"Error getting portfolio positions: {str(e)}")
            return pd.DataFrame()
    
    def get_account_summary(self) -> pd.DataFrame:
        """
        Get account summary information
        
        Returns:
            DataFrame with account summary
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return pd.DataFrame()
            
        try:
            account_values = self.ib.accountSummary()
            summary_data = []
            
            for value in account_values:
                summary_data.append({
                    'account': value.account,
                    'tag': value.tag,
                    'value': value.value,
                    'currency': value.currency
                })
            
            return pd.DataFrame(summary_data)
            
        except Exception as e:
            logger.error(f"Error getting account summary: {str(e)}")
            return pd.DataFrame()
    
    # Contract and Search Functions
    def search_contracts(self, symbol: str, sec_type: str = 'STK') -> pd.DataFrame:
        """
        Search for contracts by symbol
        
        Args:
            symbol: Symbol to search for
            sec_type: Security type (STK, OPT, FUT, etc.)
            
        Returns:
            DataFrame with matching contracts
        """
        if not self.is_connected():
            logger.error("Not connected to IBKR")
            return pd.DataFrame()
            
        try:
            # Create search contract
            contract = Contract()
            contract.symbol = symbol
            contract.secType = sec_type
            
            # Search for contracts
            contracts = self.ib.reqContractDetails(contract)
            contracts_data = []
            
            for contract_detail in contracts:
                contract = contract_detail.contract
                contracts_data.append({
                    'symbol': contract.symbol,
                    'sec_type': contract.secType,
                    'exchange': contract.exchange,
                    'currency': contract.currency,
                    'local_symbol': contract.localSymbol,
                    'trading_class': contract.tradingClass,
                    'con_id': contract.conId
                })
            
            return pd.DataFrame(contracts_data)
            
        except Exception as e:
            logger.error(f"Error searching contracts: {str(e)}")
            return pd.DataFrame()
    
    # Helper Functions
    def _create_contract(self, contract_details: ContractDetails):
        """Create IB contract from contract details"""
        if not IB_INSYNC_AVAILABLE:
            return None
            
        contract = Contract()
        contract.symbol = contract_details.symbol
        contract.secType = contract_details.sec_type
        contract.exchange = contract_details.exchange
        contract.currency = contract_details.currency
        
        if contract_details.expiry:
            contract.lastTradeDateOrContractMonth = contract_details.expiry
        if contract_details.strike:
            contract.strike = contract_details.strike
        if contract_details.right:
            contract.right = contract_details.right
        if contract_details.multiplier:
            contract.multiplier = contract_details.multiplier
            
        return contract
    
    def _create_order(self, order_details: OrderDetails):
        """Create IB order from order details"""
        if not IB_INSYNC_AVAILABLE:
            return None
            
        order = Order()
        order.action = order_details.action
        order.totalQuantity = order_details.quantity
        order.orderType = order_details.order_type
        order.tif = order_details.time_in_force
        
        if order_details.limit_price:
            order.lmtPrice = order_details.limit_price
        if order_details.stop_price:
            order.auxPrice = order_details.stop_price
        if order_details.good_till_date:
            order.goodTillDate = order_details.good_till_date
            
        return order
    
    def __enter__(self):
        """Context manager entry"""
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit"""
        self.disconnect()

# Utility Functions and Examples

In [ ]:
# Utility functions for common operations

def create_stock_contract(symbol: str, exchange: str = 'SMART', currency: str = 'USD') -> ContractDetails:
    """Create a stock contract"""
    return ContractDetails(
        symbol=symbol,
        sec_type='STK',
        exchange=exchange,
        currency=currency
    )

def create_option_contract(symbol: str, expiry: str, strike: float, right: str, 
                          exchange: str = 'SMART', currency: str = 'USD') -> ContractDetails:
    """Create an option contract"""
    return ContractDetails(
        symbol=symbol,
        sec_type='OPT',
        exchange=exchange,
        currency=currency,
        expiry=expiry,
        strike=strike,
        right=right
    )

def create_market_order(action: str, quantity: int) -> OrderDetails:
    """Create a market order"""
    return OrderDetails(
        action=action,
        quantity=quantity,
        order_type=OrderType.MARKET.value
    )

def create_limit_order(action: str, quantity: int, limit_price: float) -> OrderDetails:
    """Create a limit order"""
    return OrderDetails(
        action=action,
        quantity=quantity,
        order_type=OrderType.LIMIT.value,
        limit_price=limit_price
    )

def create_stop_order(action: str, quantity: int, stop_price: float) -> OrderDetails:
    """Create a stop order"""
    return OrderDetails(
        action=action,
        quantity=quantity,
        order_type=OrderType.STOP.value,
        stop_price=stop_price
    )

def create_stop_limit_order(action: str, quantity: int, limit_price: float, stop_price: float) -> OrderDetails:
    """Create a stop-limit order"""
    return OrderDetails(
        action=action,
        quantity=quantity,
        order_type=OrderType.STOP_LIMIT.value,
        limit_price=limit_price,
        stop_price=stop_price
    )

# Usage Examples

In [ ]:
# Example 1: Basic Connection and Market Data
def example_basic_usage():
    """Example of basic IBKR API usage"""
    
    # Create configuration (paper trading)
    config = IBKRConfig(
        host='127.0.0.1',
        port=7497,  # Paper trading port
        client_id=1
    )
    
    # Create IBKR instance
    with IBKR_Functions(config) as ibkr:
        # Connect to TWS/Gateway
        if not ibkr.connect():
            print("Failed to connect to IBKR")
            return
        
        # Create stock contract
        apple_contract = create_stock_contract('AAPL')
        
        # Get market data
        market_data = ibkr.get_market_data(apple_contract)
        if market_data:
            print(f"AAPL Market Data:")
            print(f"  Bid: {market_data.bid}")
            print(f"  Ask: {market_data.ask}")
            print(f"  Last: {market_data.last}")
            print(f"  Volume: {market_data.volume}")
        
        # Get historical data
        historical_data = ibkr.get_historical_data(apple_contract, '1 M', '1 day')
        if not historical_data.empty:
            print(f"\nHistorical Data (last 5 days):")
            print(historical_data.tail())
        
        # Get portfolio positions
        positions = ibkr.get_portfolio_positions()
        if not positions.empty:
            print(f"\nPortfolio Positions:")
            print(positions)
        
        # Get account summary
        account_summary = ibkr.get_account_summary()
        if not account_summary.empty:
            print(f"\nAccount Summary:")
            print(account_summary[account_summary['tag'].isin(['TotalCashValue', 'NetLiquidation'])])

# Example 2: Order Management
def example_order_management():
    """Example of order management"""
    
    config = IBKRConfig(port=7497)  # Paper trading
    
    with IBKR_Functions(config) as ibkr:
        if not ibkr.connect():
            print("Failed to connect to IBKR")
            return
        
        # Create contract and orders
        contract = create_stock_contract('MSFT')
        
        # Place a limit buy order
        buy_order = create_limit_order('BUY', 100, 350.00)
        order_id = ibkr.place_order(contract, buy_order)
        
        if order_id:
            print(f"Order placed with ID: {order_id}")
            
            # Wait and check order status
            time.sleep(2)
            open_orders = ibkr.get_open_orders()
            print(f"\nOpen Orders:")
            print(open_orders)
            
            # Cancel the order
            if ibkr.cancel_order(order_id):
                print(f"Order {order_id} cancelled successfully")

# Example 3: Contract Search
def example_contract_search():
    """Example of contract search"""
    
    config = IBKRConfig(port=7497)
    
    with IBKR_Functions(config) as ibkr:
        if not ibkr.connect():
            print("Failed to connect to IBKR")
            return
        
        # Search for stock contracts
        contracts = ibkr.search_contracts('GOOGL', 'STK')
        if not contracts.empty:
            print("GOOGL Stock Contracts:")
            print(contracts)
        
        # Search for option contracts
        option_contracts = ibkr.search_contracts('AAPL', 'OPT')
        if not option_contracts.empty:
            print("\nAAPL Option Contracts (first 10):")
            print(option_contracts.head(10))

# Example 4: Advanced Order Types
def example_advanced_orders():
    """Example of advanced order types"""
    
    config = IBKRConfig(port=7497)
    
    with IBKR_Functions(config) as ibkr:
        if not ibkr.connect():
            print("Failed to connect to IBKR")
            return
        
        contract = create_stock_contract('TSLA')
        
        # Stop-loss order
        stop_order = create_stop_order('SELL', 100, 200.00)
        stop_order_id = ibkr.place_order(contract, stop_order)
        
        # Stop-limit order
        stop_limit_order = create_stop_limit_order('SELL', 100, 195.00, 200.00)
        stop_limit_order_id = ibkr.place_order(contract, stop_limit_order)
        
        print(f"Stop order ID: {stop_order_id}")
        print(f"Stop-limit order ID: {stop_limit_order_id}")
        
        # Check orders
        time.sleep(2)
        open_orders = ibkr.get_open_orders()
        print("\nOpen Orders:")
        print(open_orders)

# Uncomment to run examples (requires TWS/Gateway connection)
# example_basic_usage()
# example_order_management()
# example_contract_search()
# example_advanced_orders()

print("IBKR Functions module loaded successfully!")
print("\nTo use these functions:")
print("1. Install Interactive Brokers TWS or IB Gateway")
print("2. Enable API connections in TWS/Gateway")
print("3. Install ib_insync: pip install ib_insync")
print("4. Run the example functions above")
print("\nNote: Start with paper trading for testing!")

# Integration with Existing Backtesting Engine

In [ ]:
# Integration functions to work with existing backtesting engine

def ibkr_to_eodhd_format(ibkr_data: pd.DataFrame) -> pd.DataFrame:
    """
    Convert IBKR historical data format to EODHD format for compatibility
    
    Args:
        ibkr_data: DataFrame from IBKR historical data
        
    Returns:
        DataFrame in EODHD format
    """
    if ibkr_data.empty:
        return pd.DataFrame()
    
    # Map IBKR columns to EODHD format
    eodhd_format = pd.DataFrame()
    
    if 'date' in ibkr_data.columns:
        eodhd_format['date'] = ibkr_data['date']
    
    # Map price columns
    column_mapping = {
        'open': 'open',
        'high': 'high', 
        'low': 'low',
        'close': 'close',
        'volume': 'volume'
    }
    
    for ibkr_col, eodhd_col in column_mapping.items():
        if ibkr_col in ibkr_data.columns:
            eodhd_format[eodhd_col] = ibkr_data[ibkr_col]
    
    # Add adjusted close (same as close for simplicity)
    if 'close' in eodhd_format.columns:
        eodhd_format['adjusted_close'] = eodhd_format['close']
    
    return eodhd_format

def create_live_trading_bridge(ibkr_functions: IBKR_Functions):
    """
    Create a bridge between backtesting signals and live trading
    
    Args:
        ibkr_functions: Connected IBKR_Functions instance
        
    Returns:
        Trading bridge function
    """
    
    def execute_trading_signal(symbol: str, signal: str, quantity: int, price: float = None):
        """
        Execute a trading signal from backtesting
        
        Args:
            symbol: Stock symbol
            signal: 'BUY' or 'SELL'
            quantity: Number of shares
            price: Limit price (optional, uses market order if None)
        """
        if not ibkr_functions.is_connected():
            logger.error("IBKR not connected")
            return None
        
        # Create contract
        contract = create_stock_contract(symbol)
        
        # Create order
        if price is not None:
            order = create_limit_order(signal, quantity, price)
        else:
            order = create_market_order(signal, quantity)
        
        # Place order
        order_id = ibkr_functions.place_order(contract, order)
        
        if order_id:
            logger.info(f"Signal executed: {signal} {quantity} {symbol} at {price or 'market'}")
            return order_id
        else:
            logger.error(f"Failed to execute signal: {signal} {quantity} {symbol}")
            return None
    
    return execute_trading_signal

def real_time_portfolio_monitor(ibkr_functions: IBKR_Functions, update_interval: int = 60):
    """
    Monitor portfolio in real-time
    
    Args:
        ibkr_functions: Connected IBKR_Functions instance
        update_interval: Update interval in seconds
    """
    
    def monitor_loop():
        while ibkr_functions.is_connected():
            try:
                # Get current positions
                positions = ibkr_functions.get_portfolio_positions()
                
                if not positions.empty:
                    total_value = positions['market_value'].sum()
                    total_pnl = positions['unrealized_pnl'].sum()
                    
                    print(f"\n=== Portfolio Update - {dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===")
                    print(f"Total Portfolio Value: ${total_value:,.2f}")
                    print(f"Total Unrealized P&L: ${total_pnl:,.2f}")
                    print(f"Number of Positions: {len(positions)}")
                    
                    # Show top positions
                    if len(positions) > 0:
                        print("\nTop Positions:")
                        top_positions = positions.nlargest(5, 'market_value')[['symbol', 'position', 'market_value', 'unrealized_pnl']]
                        print(top_positions.to_string(index=False))
                
                time.sleep(update_interval)
                
            except Exception as e:
                logger.error(f"Error in portfolio monitoring: {str(e)}")
                time.sleep(update_interval)
    
    # Start monitoring in a separate thread
    monitor_thread = threading.Thread(target=monitor_loop, daemon=True)
    monitor_thread.start()
    
    return monitor_thread

# Example: Complete workflow from backtesting to live trading
def complete_trading_workflow_example():
    """
    Example of complete workflow from backtesting to live trading
    """
    
    # Step 1: Connect to IBKR
    config = IBKRConfig(port=7497)  # Paper trading
    
    with IBKR_Functions(config) as ibkr:
        if not ibkr.connect():
            print("Failed to connect to IBKR")
            return
        
        # Step 2: Get live data for analysis
        symbols = ['AAPL', 'MSFT', 'GOOGL']
        live_data = {}
        
        for symbol in symbols:
            contract = create_stock_contract(symbol)
            historical_data = ibkr.get_historical_data(contract, '30 D', '1 day')
            
            if not historical_data.empty:
                # Convert to EODHD format for compatibility
                eodhd_format = ibkr_to_eodhd_format(historical_data)
                live_data[symbol] = eodhd_format
        
        # Step 3: Run analysis (simplified example)
        signals = []
        for symbol, data in live_data.items():
            if len(data) > 20:  # Simple moving average crossover
                data['ma_5'] = data['close'].rolling(5).mean()
                data['ma_20'] = data['close'].rolling(20).mean()
                
                if data['ma_5'].iloc[-1] > data['ma_20'].iloc[-1]:
                    signals.append({'symbol': symbol, 'action': 'BUY', 'quantity': 10})
                elif data['ma_5'].iloc[-1] < data['ma_20'].iloc[-1]:
                    signals.append({'symbol': symbol, 'action': 'SELL', 'quantity': 10})
        
        # Step 4: Execute signals
        trading_bridge = create_live_trading_bridge(ibkr)
        
        for signal in signals:
            order_id = trading_bridge(
                signal['symbol'], 
                signal['action'], 
                signal['quantity']
            )
            print(f"Signal executed: {signal} -> Order ID: {order_id}")
        
        # Step 5: Start portfolio monitoring
        monitor_thread = real_time_portfolio_monitor(ibkr, 30)
        
        print("\nTrading workflow completed. Portfolio monitoring started.")
        print("Press Ctrl+C to stop monitoring...")
        
        # Keep the program running
        try:
            time.sleep(300)  # Run for 5 minutes
        except KeyboardInterrupt:
            print("\nMonitoring stopped.")

# Uncomment to run the complete workflow example
# complete_trading_workflow_example()

print("\nIntegration functions loaded successfully!")
print("These functions help bridge backtesting results with live trading execution.")

# Documentation and Best Practices

## Setup Instructions

### 1. Install Interactive Brokers Software
- Download and install TWS (Trader Workstation) or IB Gateway
- Create an Interactive Brokers account
- Enable API access in account settings

### 2. Configure API Settings
- In TWS/Gateway, go to File > Global Configuration > API > Settings
- Enable "Enable ActiveX and Socket Clients"
- Set Socket port (7497 for paper trading, 7496 for live)
- Add trusted IP addresses (127.0.0.1 for local)

### 3. Install Python Dependencies
```bash
pip install ib_insync pandas numpy
```

## Usage Patterns

### Basic Connection
```python
# Paper trading configuration
config = IBKRConfig(port=7497)
with IBKR_Functions(config) as ibkr:
    if ibkr.connect():
        # Your trading logic here
        pass
```

### Market Data
```python
# Get real-time market data
contract = create_stock_contract('AAPL')
market_data = ibkr.get_market_data(contract)

# Get historical data
historical_data = ibkr.get_historical_data(contract, '1 Y', '1 day')
```

### Order Management
```python
# Place a limit order
order = create_limit_order('BUY', 100, 150.00)
order_id = ibkr.place_order(contract, order)

# Cancel an order
ibkr.cancel_order(order_id)
```

## Best Practices

### 1. Risk Management
- Always use paper trading for testing
- Implement position sizing rules
- Set stop-loss orders
- Monitor portfolio exposure

### 2. Error Handling
- Always check connection status
- Implement retry logic for API calls
- Log all trading activities
- Handle market closure periods

### 3. Performance
- Limit API call frequency
- Use efficient data structures
- Implement proper logging
- Monitor system resources

### 4. Security
- Secure API credentials
- Use IP whitelisting
- Implement authentication
- Regular security audits

## Common Issues and Solutions

### Connection Issues
- Check TWS/Gateway is running
- Verify API settings are enabled
- Ensure correct port numbers
- Check firewall settings

### Market Data Issues
- Verify market data subscriptions
- Check market hours
- Use appropriate data types
- Handle delayed data permissions

### Order Issues
- Check account permissions
- Verify contract details
- Ensure sufficient buying power
- Handle order rejections

## Integration with Existing Code

This IBKR implementation is designed to work alongside the existing EODHD functions:

1. **Data Compatibility**: Historical data can be converted to EODHD format
2. **Backtesting Integration**: Signals from backtesting can be executed live
3. **Portfolio Management**: Real-time monitoring of live positions
4. **Risk Management**: Integration with existing risk models

## Support and Resources

- [Interactive Brokers API Documentation](https://interactivebrokers.github.io/tws-api/)
- [ib_insync Documentation](https://ib-insync.readthedocs.io/)
- [TWS API Reference](https://interactivebrokers.github.io/tws-api/basic_contracts.html)

## Disclaimer

This code is for educational and research purposes. Trading involves risk of loss. Always:
- Test thoroughly in paper trading
- Understand the risks involved
- Consult with financial advisors
- Follow regulatory requirements
- Use appropriate risk management